In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Importing the train data

In [3]:
from proj1_helpers import *
#DATA_TRAIN_PATH = '/Users/alperkose/Desktop/train.csv' # TODO: download train data and supply path here 
DATA_TRAIN_PATH = 'train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
def split_data(x, y, ratio,seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data based on the given ratio: TODO
    
    c = list(zip(x, y))

    np.random.shuffle(c)

    x, y = zip(*c)
    
    ind=round(ratio*len(x))
    ind2=len(x)
    x=np.array(x)
    trainx = x[0:ind,:]
    trainy = y[0:ind]
    testx = x[ind:ind2,:]
    testy = y[ind:ind2]
    
    return trainx, testx,trainy,testy

In [7]:
#tX=np.delete(tX,[0,4,5,6,12,23,24,25,26,27,28],axis=1)

# Useful line ?
tX[tX==-999] = 0

# Removing all of the fucked up -999 and centering the distribution
# Same with continuous / non continuous values ?
m = np.mean(tX, axis=0)
centered_tX = tX - m
centered_tX[tX==-999] = 0

# Setting them to nan in order to compute the std without having to 
# worry about the outliers. Putting them back to 0 after.
centered_tX[centered_tX==0] = float('nan')
stdevtrain = np.nanstd(centered_tX, axis=0);
centered_tX[centered_tX==float('nan')] = 0

standardized_tX = centered_tX / stdevtrain

# Splitting the sizes of features and samples
d = len(standardized_tX[0])
n = len(standardized_tX)
print(d, n)
new_dims = 0

for t in range (1, d + 1):
    new_dims = new_dims + t * (t + 1) / 2
  
print(new_dims)
mat = np.zeros((n, new_dims + int(d * (d + 1) / 2 + d)))
print(mat.shape)
q = 0
tao = d 

for i in range (n):
    for t in range (d):  
        mat[i, t] = standardized_tX[i, t]


        
for i in range (n):
    for monsoon in range (d):
        for t in range (monsoon,d):
            mat[i, tao] = standardized_tX[i, t] * standardized_tX[i,q]
            tao=tao+1
        q=q+1
    tao=d
    q=0

tao = int(d * (d + 1) / 2 + d)
    
for i in range (n):
    for r in range (d):
        for monsoon in range (r,d):
            for t in range (monsoon,d): 
                mat[i, tao] = standardized_tX[i, t] * standardized_tX[i, monsoon] * standardized_tX[i, r]
                tao = tao + 1
    tao=int(d * (d + 1) / 2 + d)




30 250000
4960.0
(250000, 5455)


/home/romain/.local/lib/python3.5/site-packages/ipykernel/__main__.py:30: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
num_samples=len(standardized_tX)
tx = np.c_[np.ones(num_samples), mat]
#y=list(y)
#y[y==-1]=0

In [ ]:
def compute_loss(y, tx, w):
    """Calculate the loss.
    
    You can calculate the loss using mse or mae.
    """
    # ***************************************************
    e=y-np.dot(tx,w)
    mse=np.dot(e.transpose(),e)/(2*len(tx))
    return mse
    # ***************************************************
    raise NotImplementedError

In [ ]:
tX,testx,y,testy=split_data(tx, y, 0.8,seed=1)

In [ ]:
from __future__ import division
import numpy as np 
def logistic(a):
    return 1.0 / (1 + np.exp(-a))
def irls(X, y):
    theta = np.zeros(X.shape[1])
    theta_ = np.inf
    eps=250000
    lamda=10
    for aqua in range (3):
        grad=np.zeros(X.shape[1])
        a = np.dot(X, theta)
        pi = logistic(a)        
        SX = X * (pi - pi*pi).reshape(-1,1)
        XSX = np.dot(X.T,SX)+lamda*np.eye((len(X[0])))
        #+lamda*np.eye((len(X[0])))
        for aw in range (len(tx)):
            grad=grad+(-1/len(tx))*(y[aw]*X[aw,:]*logistic(-y[aw]*np.dot(X[aw,:],theta)))
        
        
     
        theta = theta-eps*np.linalg.solve(XSX,grad)
        print(sum(y==np.sign(np.dot(X,theta)))/len(y))
       
        if aqua%10==0 and aqua!=0:
            eps=eps*0.2
    return theta

In [ ]:
theta=irls(tX,y)

In [ ]:
#testx=np.delete(testx,[0,4,5,6,12,23,24,25,26,27,28],axis=1)
testx[testx==-999]=0
m2=np.mean(testx,axis=0)
centered_testx=testx-m2
centered_testx[testx==-999]=0
standardized_testx=centered_testx / np.std(centered_testx, axis=0)
centered_testx[centered_testx==0]=float('nan')
stdevtest=np.nanstd(centered_testx,axis=0);
centered_testx[centered_testx==float('nan')]=0
standardized_testx=centered_testx / stdevtest

d=len(standardized_testx[0])
n=len(standardized_testx)
mat=np.zeros((n,new_dims+int(d*(d+1)/2+d)))
q=0
tao=d 

for i in range (n):
    for t in range (d):  
        mat[i,t]=standardized_testx[i,t]

        
for i in range (n):
    for mk in range (d):
        for t in range (mk,d): 
            mat[i,tao]=standardized_testx[i,t]*standardized_testx[i,q]
            tao=tao+1
        q=q+1    
    tao=d  
    q=0

tao=int(d*(d+1)/2+d)
    
for i in range (n):
    for r in range (d):
        for monsoon in range (r,d):
            for t in range (monsoon,d): 
                mat[i,tao]=standardized_testx[i,t]*standardized_testx[i,monsoon]*standardized_testx[i,r]
                tao=tao+1
    tao=int(d*(d+1)/2+d)  

    
num_samples=len(standardized_testx)
final_testx = np.c_[np.ones(num_samples), mat]



In [ ]:
aq=np.dot(testx,theta)
y_guess=np.sign(aq)
trueness=sum((y_guess==testy))/len(testy)
print(trueness)

In [ ]:
theta

In [ ]:
aq=sum(tX.transpose()==-999)
aq[1:100]

In [ ]:
y[1:100]

In [ ]:
sum(testx==-999)